In [1]:
import baostock as bs
import pandas as pd

In [2]:
# 登录系统
lg = bs.login()

login success!


# 筛选高波动、且具有上行空间的股票

In [36]:
# 登录Baostock
bs.login()

# 获取所有A股股票列表
stock_rs = bs.query_all_stock(day="2023-11-10")
stock_df = stock_rs.get_data()

# 剔除北交所和创业板股票
stock_df = stock_df[~stock_df['code'].str.startswith(('bj', 'sz.3'))]

# 退出Baostock
bs.logout()

login success!
logout success!


In [39]:
print(stock_df['code'])
print(stock_df['code'])

229     sh.000001
230     sh.000002
231     sh.000003
232     sh.000004
233     sh.000005
          ...    
4193    sz.003040
4194    sz.003041
4195    sz.003042
4196    sz.003043
4197    sz.003816
Name: code, Length: 3969, dtype: object
229     sh.000001
230     sh.000002
231     sh.000003
232     sh.000004
233     sh.000005
          ...    
4193    sz.003040
4194    sz.003041
4195    sz.003042
4196    sz.003043
4197    sz.003816
Name: code, Length: 3969, dtype: object


In [40]:
from datetime import datetime, timedelta

# 设置日期范围
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')

# 登录Baostock
bs.login()

# 创建一个空的DataFrame用于存储结果
selected_stocks = pd.DataFrame(dict.fromkeys(['code', 'monthly_volatility', 'recent_volatility_increase'],[]))

for stock in stock_df['code']:
    # 获取股票日K线数据
    k_rs = bs.query_history_k_data_plus(stock, "date,close",
                                        start_date=start_date, end_date=end_date,
                                        frequency="d", adjustflag="3")
    k_data = k_rs.get_data()
    
    # 检查'close'列是否存在
    if 'close' in k_data.columns:
    # 计算日收益率
        k_data['close'] = k_data['close'].astype(float)
        k_data['daily_return'] = k_data['close'].pct_change()

        # 计算近一个月的波动率
        monthly_volatility = k_data['daily_return'].std() * (252**0.5)  # 年化波动率

        # 计算近3天波动率
        recent_volatility = k_data['daily_return'][-3:].std() * (252**0.5)  # 年化波动率

        # 判断近3天的波动是否显著大于近一个月的波动
        if recent_volatility > 1.5 * monthly_volatility:
            selected_stocks.loc[len(selected_stocks)]=[stock, monthly_volatility, True]
    else:
        # 如果不存在'close'列，可能需要记录下来或采取其他措施
        print(f"Data for stock {stock} does not have 'close' column.")

# 退出Baostock
bs.logout()

login success!
logout success!


In [41]:
print(selected_stocks.sort_values(by="monthly_volatility",ascending=False))

          code  monthly_volatility  recent_volatility_increase
177  sz.002855            1.172487                        True
65   sh.603496            1.050614                        True
53   sh.603158            1.037994                        True
181  sz.002931            1.020544                        True
49   sh.603003            0.983502                        True
..         ...                 ...                         ...
48   sh.601929            0.198454                        True
136  sz.002005            0.195715                        True
139  sz.002102            0.180053                        True
170  sz.002726            0.176094                        True
0    sh.000012            0.004660                        True

[184 rows x 3 columns]


In [5]:
# 获取历史K线数据
stock_code = "sh.600000"  # 示例股票代码
start_date = '2020-01-01'
end_date = '2020-12-31'
data_list = []

rs = bs.query_history_k_data_plus(stock_code,
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST",
    start_date=start_date, end_date=end_date,
    frequency="d", adjustflag="3")

while (rs.error_code == '0') & rs.next():
    # 将数据添加到列表中
    data_list.append(rs.get_row_data())

# 转换为 DataFrame
data_df = pd.DataFrame(data_list, columns=rs.fields)

# 登出系统
bs.logout()

data_df.to_csv('stock_data.csv', index=False)

logout success!
